In [ ]:
%load_ext autoreload
%autoreload 2

# DNS Seeds

By this time you've probably hit this error:

![image](../images/empty-list.png)

It would be much better if we could prime our crawler with more addresses. It would be especially good if they were addresses of "high quality" nodes that are always online and have juicy peer lists to share with us.

This is exactly what DNS seeds are for. Prominent bitcoin core developers run DNS servers from domains they contol which resolve not to the address of a machine serving a website -- which is true of most domain names -- but to a list of addresses of high quality bitcoin full nodes.

These domains are actually [hard-coded into Bitcoin Core](https://github.com/bitcoin/bitcoin/blob/v0.17.1/src/chainparams.cpp#L127)!

Some of them run [this crawler / server written by Peter Wuille](https://github.com/sipa/bitcoin-seeder).

Let's learn to query these DNS seeds:

In your terminal type:

```shell
$ dig dnsseed.bitcoin.dashjr.org
```

This will perform a DNS lookup. We are interest in the "answers" sections, which shows a number of DNS "A records" for the host name `dnsseed.bitcoin.dashjr.org`. These are IP addresses of Bitcoin nodes!

![image](../images/dig.png)


How can we do this from Python?

Python's built-in [`socket.getaddrinfo`](https://docs.python.org/3/library/socket.html#socket.getaddrinfo) function can accomplish this.

In [ ]:
# Copied from Bitcoin Core's src/chainparams.cpp file

DNS_SEEDS = [
    'dnsseed.bitcoin.dashjr.org', 
    'dnsseed.bluematt.me',
    'seed.bitcoin.sipa.be', 
    'seed.bitcoinstats.com',
    'seed.bitcoin.jonasschnelli.ch',
    'seed.btc.petertodd.org',
    'seed.bitcoin.sprovoost.nl',
    'dnsseed.emzy.de',
]

In [ ]:
import socket

# getaddrinfo translates hostname -> ip address ... but it's messy
addr_info = socket.getaddrinfo(DNS_SEEDS[0], 8333)
addr_info

In [ ]:
# Third param can filter down IPv4 or IPv6

# IPv4 example
socket.getaddrinfo(DNS_SEEDS[0], 8333, socket.AF_INET)

In [ ]:
# IPv6 example
socket.getaddrinfo(DNS_SEEDS[0], 8333, socket.AF_INET6)

In [ ]:
# 0 accepts both (what we want)
socket.getaddrinfo(DNS_SEEDS[0], 8333, 0)

In [ ]:
# 4th paramter sets the socket type (TCP, UDP, or RAW)

socket.getaddrinfo(DNS_SEEDS[0], 8333, 0, socket.SOCK_STREAM)

In [ ]:
# Question: how would you getch the addresses configured for UDP?

socket.getaddrinfo(DNS_SEEDS[0], 8333, socket.AF_INET, ???)

In [ ]:
# We want to accept IPv4 & IPv6 (3rd param set to 0)
# But only TCP socket types (4th param set to socket.SOCK_STREAM)
addr_info = socket.getaddrinfo(DNS_SEEDS[0], 8333, 0, socket.SOCK_STREAM)
addr_info

In [ ]:
# addresses in the last entry
addrs = [ai[-1] for ai in addr_info]
addrs

In [ ]:
# addresses in the last entry
addrs = [ai[-1][:2] for ai in addr_info]
addrs

In [ ]:
from mycrawler import Node

# Turn them into Node instances
nodes = [Node(*addr) for addr in addrs]
nodes

In [ ]:
from mycrawler import Connection

# Can we connect?
conn = Connection(nodes[0]).open()

In [ ]:
def query_dns_seeds():
    nodes = []
    for seed in DNS_SEEDS:
        try:
            addr_info = socket.getaddrinfo(seed, 8333, 0, socket.SOCK_STREAM)
            addresses = [ai[-1][:2] for ai in addr_info]
            nodes.extend([Node(*addr) for addr in addresses])
        except OSError as e:
            logger.info(f"DNS seed query failed: {str(e)}")
    return nodes

In [ ]:
# BEHOLD THE GLORIOUS BITCOIN NODES!!!

for node in query_dns_seeds():
    print(node.ip)

BTW -- we're already using `getaddrinfo` under the hood when we call `socket.create_connection` ...